In [2]:
import csv
import numpy as np
from PIL import Image
import os, os.path
from sklearn.model_selection import train_test_split
import operator

inputs = []
train_Y = []

print('Preprocessing Images')
csvfile = open('../train.csv', 'r')
csvreader = csv.reader(csvfile)
data = [line[:3] for line in csvreader]
data = data[1:]
data = sorted(data, key=operator.itemgetter(0))
i = 0

path = '../training-images'
for filename in os.listdir(path):
    entry = int(data[i][2])
    if entry > 150 or entry < 0:
        i = i + 1
        continue

    try:
        im = Image.open(os.path.join(path, filename)).convert('L')
        im_data = list(im.getdata())
        arr = np.array(im)
        #arr = arr[:,:,0]
        arr = arr.astype('float16')
        arr = arr / 255.
        inputs.append(im_data)
        train_Y.append(entry)
        i = i + 1
    except Exception as e:
        print('Can not identify file %s' %(filename))
        i = i + 1

train_X = np.array(inputs)
print("Transforming")
#train_X = train_X.reshape(-1, 256, 256, 1)

train_X_1,test_X,train_label_1,test_label = train_test_split(train_X, train_Y, test_size=0.2, random_state=42)
train_X_2,valid_X,train_label,valid_label = train_test_split(train_X_1, train_label_1, test_size=0.2, random_state=42)

from sklearn.decomposition import PCA
pca = PCA(0.99)
X_pca_train = pca.fit_transform(train_X_2)
X_pca_test = pca.transform(test_X)
X_pca_valid = pca.transform(valid_X)

print(X_pca_valid.shape)

from sklearn.datasets import dump_svmlight_file
dump_svmlight_file(X_pca_train,train_label,'150train-30apr.input', zero_based=True, multilabel=False)
dump_svmlight_file(X_pca_valid,valid_label,'150valid-30apr.input', zero_based=True, multilabel=False)
dump_svmlight_file(X_pca_test,test_label,'150test-30apr.input', zero_based=True, multilabel=False)
print("Done!")

Preprocessing Images
Transforming
(1199, 3313)
Done!
